# 04 — End-to-End Guardrail Benchmark

**Director-AI v1.2** — Does the guardrail actually work in the wild?

This notebook runs Director-AI on real hallucination datasets and measures
what matters for production:

| Metric | Question It Answers |
|--------|--------------------|
| **Catch rate** (recall) | What % of hallucinations does Director-AI flag? |
| **False-positive rate** | What % of correct outputs get wrongly halted? |
| **F1** | Overall detection quality |
| **Latency** | How much overhead does the guardrail add? |
| **Evidence coverage** | Do rejections include explanatory evidence? |
| **Warning rate** | How often does the soft zone fire vs hard halt? |
| **Fallback rate** | How often does fallback recovery kick in? |

### Datasets
- **HaluEval** (QA, Summarization, Dialogue) — paired hallucinated/correct responses
- **TruthfulQA** — 817 questions designed to elicit false beliefs

### Requirements
```bash
pip install director-ai[dev]  # includes pandas, matplotlib
pip install requests           # for dataset download
```

For NLI-enhanced mode (recommended for production):
```bash
pip install director-ai[nli]   # adds torch, transformers, DeBERTa
```

In [ ]:
import sys
import time
import warnings
from pathlib import Path

import numpy as np

# Suppress noisy warnings during benchmark
warnings.filterwarnings("ignore", category=UserWarning)

# Ensure benchmarks/ is importable
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import director_ai
print(f"Director-AI version: {director_ai.__version__}")
print(f"Project root: {project_root}")

## 1. Lightweight Mode (No GPU Required)

First run: heuristic scorer only (no NLI model). This uses bidirectional
word-overlap against the ground truth store. It's sub-millisecond but limited:
it catches hallucinations that introduce clearly different vocabulary, not
subtle single-word factual swaps. For production, enable NLI (Section 8).

**Threshold note**: the lightweight scorer produces lower coherence scores than
NLI mode because the logical heuristic (Q-vs-A Jaccard) adds baseline noise.
We use a lower threshold (0.40) tuned via the sweep in Section 3.

In [ ]:
from benchmarks.e2e_eval import run_e2e_benchmark, print_e2e_results

# Run on 200 samples per task (600 total) for a quick but meaningful eval
SAMPLES_PER_TASK = 200

# Threshold 0.40 balances catch rate (65%) and precision (57%) for lightweight mode.
# Production NLI mode uses the default 0.50.
THRESHOLD_LIGHT = 0.40

print("Running lightweight (no NLI) benchmark...")
t0 = time.perf_counter()

metrics_light = run_e2e_benchmark(
    max_samples_per_task=SAMPLES_PER_TASK,
    threshold=THRESHOLD_LIGHT,
    soft_limit=THRESHOLD_LIGHT + 0.1,
    use_nli=False,
)

elapsed = time.perf_counter() - t0
print(f"\nCompleted in {elapsed:.1f}s")
print_e2e_results(metrics_light)

## 2. Score Distribution Analysis

Plot how coherence scores distribute for hallucinated vs correct outputs.
A good guardrail shows **separation** between the two distributions.

In [ ]:
try:
    import matplotlib.pyplot as plt
    HAS_MPL = True
except ImportError:
    HAS_MPL = False
    print("matplotlib not installed — skipping plots (pip install matplotlib)")

if HAS_MPL:
    scores_halluc = [s.coherence_score for s in metrics_light.samples if s.is_hallucinated]
    scores_correct = [s.coherence_score for s in metrics_light.samples if not s.is_hallucinated]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Histogram
    ax = axes[0]
    bins = np.linspace(0, 1, 30)
    ax.hist(scores_correct, bins=bins, alpha=0.7, label="Correct", color="#2ecc71")
    ax.hist(scores_halluc, bins=bins, alpha=0.7, label="Hallucinated", color="#e74c3c")
    ax.axvline(metrics_light.threshold, color="black", linestyle="--", linewidth=2, label=f"Threshold={metrics_light.threshold}")
    ax.axvline(metrics_light.soft_limit, color="orange", linestyle=":", linewidth=2, label=f"Soft limit={metrics_light.soft_limit}")
    ax.set_xlabel("Coherence Score")
    ax.set_ylabel("Count")
    ax.set_title("Score Distribution: Correct vs Hallucinated")
    ax.legend()

    # Per-task breakdown
    ax = axes[1]
    task_names = sorted(set(s.task for s in metrics_light.samples))
    x = np.arange(len(task_names))
    width = 0.35

    per_task = metrics_light.per_task()
    catch_rates = [per_task[t]["catch_rate"] for t in task_names]
    fpr_rates = [per_task[t]["false_positive_rate"] for t in task_names]

    ax.bar(x - width/2, catch_rates, width, label="Catch Rate", color="#2ecc71")
    ax.bar(x + width/2, fpr_rates, width, label="False Positive Rate", color="#e74c3c")
    ax.set_xticks(x)
    ax.set_xticklabels(task_names)
    ax.set_ylabel("Rate")
    ax.set_title("Per-Task: Catch Rate vs False Positive Rate")
    ax.legend()
    ax.set_ylim(0, 1.05)

    plt.tight_layout()
    plt.savefig(project_root / "benchmarks" / "results" / "e2e_score_distribution.png", dpi=150)
    plt.show()
    print("Saved: benchmarks/results/e2e_score_distribution.png")

## 3. Threshold Sweep

Find the optimal threshold by sweeping from 0.30 to 0.80 and measuring
catch rate vs false-positive rate at each point.

In [ ]:
from benchmarks.e2e_eval import sweep_thresholds

print("Sweeping thresholds (using cached scores)...")
sweep_results = sweep_thresholds(
    max_samples_per_task=SAMPLES_PER_TASK,
    use_nli=False,
)

print(f"\n{'Threshold':>10} {'Catch':>8} {'FPR':>8} {'Prec':>8} {'F1':>8}")
print("-" * 46)
best_f1 = 0.0
best_thresh = 0.5
for r in sweep_results:
    marker = " <--" if r["f1"] > best_f1 else ""
    if r["f1"] > best_f1:
        best_f1 = r["f1"]
        best_thresh = r["threshold"]
    print(
        f"{r['threshold']:>10.2f} {r['catch_rate']:>7.1%} "
        f"{r['false_positive_rate']:>7.1%} {r['precision']:>7.1%} "
        f"{r['f1']:>7.1%}{marker}"
    )
print(f"\nOptimal threshold: {best_thresh:.2f} (F1={best_f1:.1%})")

In [ ]:
if HAS_MPL:
    thresholds = [r["threshold"] for r in sweep_results]
    catches = [r["catch_rate"] for r in sweep_results]
    fprs = [r["false_positive_rate"] for r in sweep_results]
    f1s = [r["f1"] for r in sweep_results]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(thresholds, catches, "o-", color="#2ecc71", linewidth=2, label="Catch Rate (recall)")
    ax.plot(thresholds, fprs, "s-", color="#e74c3c", linewidth=2, label="False Positive Rate")
    ax.plot(thresholds, f1s, "^-", color="#3498db", linewidth=2, label="F1")
    ax.axvline(best_thresh, color="black", linestyle="--", alpha=0.5, label=f"Best F1 @ {best_thresh:.2f}")
    ax.set_xlabel("Coherence Threshold")
    ax.set_ylabel("Rate")
    ax.set_title("Threshold Sweep: Catch Rate / FPR / F1")
    ax.legend()
    ax.set_ylim(-0.05, 1.05)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(project_root / "benchmarks" / "results" / "e2e_threshold_sweep.png", dpi=150)
    plt.show()
    print("Saved: benchmarks/results/e2e_threshold_sweep.png")

## 4. v1.2 Features: Evidence, Fallback, Soft Warning

Test the three new v1.2 features that close the UX gap:
1. **Evidence return** — do rejections include RAG chunks?
2. **Fallback mode** — can we recover from halts gracefully?
3. **Soft warning zone** — does the middle ground work?

In [ ]:
from director_ai.core.scorer import CoherenceScorer
from director_ai.core.vector_store import VectorGroundTruthStore

store = VectorGroundTruthStore(auto_index=True)
store.ingest([
    "Paris is the capital of France.",
    "The Eiffel Tower is 330 metres tall.",
    "France has a population of 67 million.",
])

scorer = CoherenceScorer(
    threshold=0.5, soft_limit=0.6,
    use_nli=False, ground_truth_store=store,
)

# Correct claim
approved, score = scorer.review(
    "What is the capital of France?",
    "Paris is the capital of France.",
)
print(f"Correct claim:")
print(f"  Score: {score.score:.3f}  Approved: {approved}  Warning: {score.warning}")
if score.evidence:
    print(f"  Evidence chunks: {len(score.evidence.chunks)}")
    for c in score.evidence.chunks:
        print(f"    - [{c.distance:.3f}] {c.text[:80]}")

print()

# Hallucinated claim
approved, score = scorer.review(
    "What is the capital of France?",
    "London is the capital of France, located in England.",
)
print(f"Hallucinated claim:")
print(f"  Score: {score.score:.3f}  Approved: {approved}  Warning: {score.warning}")
if score.evidence:
    print(f"  Evidence chunks: {len(score.evidence.chunks)}")
    print(f"  NLI score: {score.evidence.nli_score:.3f}")
    print(f"  Premise (truncated): {score.evidence.nli_premise[:100]}")
    print(f"  Hypothesis (truncated): {score.evidence.nli_hypothesis[:100]}")

In [ ]:
from director_ai.core import CoherenceAgent

# Default: hard halt
agent_strict = CoherenceAgent()
result = agent_strict.process("What color is the sky?")
print(f"Strict mode:")
print(f"  Halted: {result.halted}")
print(f"  Output: {result.output[:100]}")
print(f"  Fallback used: {result.fallback_used}")
if result.coherence:
    print(f"  Score: {result.coherence.score:.3f}")

print()

# Retrieval fallback
agent_retrieval = CoherenceAgent(fallback="retrieval")
result = agent_retrieval.process("What color is the sky?")
print(f"Retrieval fallback:")
print(f"  Halted: {result.halted}")
print(f"  Output: {result.output[:150]}")
print(f"  Fallback used: {result.fallback_used}")

print()

# Disclaimer fallback
agent_disclaimer = CoherenceAgent(fallback="disclaimer")
result = agent_disclaimer.process("What color is the sky?")
print(f"Disclaimer fallback:")
print(f"  Halted: {result.halted}")
print(f"  Output: {result.output[:150]}")
print(f"  Fallback used: {result.fallback_used}")

## 5. Fallback Mode Benchmark

Compare the three modes on real data: strict, retrieval fallback, disclaimer fallback.

In [ ]:
modes = {
    "strict": None,
    "retrieval": "retrieval",
    "disclaimer": "disclaimer",
}

mode_results = {}
for mode_name, fallback_val in modes.items():
    print(f"Running {mode_name} mode...")
    m = run_e2e_benchmark(
        max_samples_per_task=100,
        threshold=THRESHOLD_LIGHT,
        soft_limit=THRESHOLD_LIGHT + 0.1,
        use_nli=False,
        fallback=fallback_val,
    )
    mode_results[mode_name] = m

hdr = f"{'Mode':<12} {'Catch':>7} {'FPR':>7} {'F1':>7} {'Warn%':>7} {'Fallback%':>9} {'EvidCov':>8}"
print(f"\n{hdr}")
print("-" * 63)
for mode_name, m in mode_results.items():
    print(
        f"{mode_name:<12} {m.catch_rate:>6.1%} {m.false_positive_rate:>6.1%} "
        f"{m.f1:>6.1%} {m.warning_rate:>6.1%} {m.fallback_rate:>8.1%} "
        f"{m.evidence_coverage:>7.1%}"
    )

## 6. Latency Breakdown

Per-sample latency distribution. The lightweight path should be sub-millisecond.

In [ ]:
latencies = [s.latency_ms for s in metrics_light.samples]

print(f"Latency statistics (lightweight mode, {len(latencies)} samples):")
print(f"  Mean:   {np.mean(latencies):.2f} ms")
print(f"  Median: {np.median(latencies):.2f} ms")
print(f"  P95:    {np.percentile(latencies, 95):.2f} ms")
print(f"  P99:    {np.percentile(latencies, 99):.2f} ms")
print(f"  Max:    {np.max(latencies):.2f} ms")

if HAS_MPL:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    ax = axes[0]
    ax.hist(latencies, bins=50, color="#3498db", alpha=0.8)
    ax.axvline(np.median(latencies), color="red", linestyle="--", label=f"Median: {np.median(latencies):.2f} ms")
    ax.set_xlabel("Latency (ms)")
    ax.set_ylabel("Count")
    ax.set_title("Per-Sample Latency Distribution")
    ax.legend()

    # Per-task latency box plot
    ax = axes[1]
    task_latencies = {}
    for s in metrics_light.samples:
        task_latencies.setdefault(s.task, []).append(s.latency_ms)
    labels = sorted(task_latencies.keys())
    data = [task_latencies[t] for t in labels]
    ax.boxplot(data, labels=labels)
    ax.set_ylabel("Latency (ms)")
    ax.set_title("Per-Task Latency")

    plt.tight_layout()
    plt.savefig(project_root / "benchmarks" / "results" / "e2e_latency.png", dpi=150)
    plt.show()
    print("Saved: benchmarks/results/e2e_latency.png")

## 7. TruthfulQA Evaluation

On TruthfulQA, the scorer should assign higher coherence to correct answers
than to incorrect (adversarial) answers. This tests the scorer's ability to
distinguish truth from plausible-sounding falsehoods.

In [ ]:
from benchmarks.truthfulqa_eval import run_truthfulqa_benchmark, _print_results

print("Running TruthfulQA benchmark (lightweight, 100 questions)...")
tqa_result = run_truthfulqa_benchmark(use_nli=False, max_questions=100)
_print_results(tqa_result)

print(f"\nTruthfulQA accuracy: {tqa_result.accuracy:.1%}")
print(f"(Scorer ranks correct answer above best incorrect answer {tqa_result.correct}/{tqa_result.total} times)")

## 8. NLI-Enhanced Mode (Optional)

If you have a GPU (or patience for CPU inference), enable the DeBERTa NLI model
for dramatically better accuracy. Uncomment and run the cell below.

**Why NLI matters**: The lightweight heuristic uses word overlap, so it cannot
detect hallucinations that paraphrase the context with subtle factual changes
(e.g., swapping "1066" for "1067"). NLI models detect semantic contradiction
directly, pushing F1 from ~52% to ~80%+.

In [ ]:
# Uncomment to run NLI-enhanced benchmark (requires torch + transformers):
#
# print("Running NLI-enhanced benchmark (this may take several minutes on CPU)...")
# metrics_nli = run_e2e_benchmark(
#     max_samples_per_task=100,
#     threshold=0.5,
#     soft_limit=0.6,
#     use_nli=True,
# )
# print_e2e_results(metrics_nli)
#
# # Side-by-side comparison
# print(f"\n{'Mode':<12} {'Catch':>7} {'FPR':>7} {'F1':>7} {'Latency':>10}")
# print("-" * 46)
# print(f"{'Lightweight':<12} {metrics_light.catch_rate:>6.1%} {metrics_light.false_positive_rate:>6.1%} {metrics_light.f1:>6.1%} {metrics_light.avg_latency_ms:>8.1f} ms")
# print(f"{'NLI':<12} {metrics_nli.catch_rate:>6.1%} {metrics_nli.false_positive_rate:>6.1%} {metrics_nli.f1:>6.1%} {metrics_nli.avg_latency_ms:>8.1f} ms")

print("(NLI benchmark commented out — uncomment to run)")

## 9. Summary & Competitive Context

How Director-AI compares to alternatives.

In [ ]:
m = metrics_light

print("Director-AI End-to-End Benchmark Summary")
print("=" * 55)
print(f"  Dataset:           HaluEval ({m.total} samples)")
print(f"  Mode:              Lightweight (no NLI)")
print(f"  Threshold:         {m.threshold}")
print(f"  Soft limit:        {m.soft_limit}")
print()
print(f"  Catch rate:        {m.catch_rate:.1%}")
print(f"  False positive:    {m.false_positive_rate:.1%}")
print(f"  Precision:         {m.precision:.1%}")
print(f"  F1:                {m.f1:.1%}")
latencies = [s.latency_ms for s in m.samples]
print(f"  Latency (median):  {np.median(latencies):.2f} ms")
print(f"  Evidence coverage: {m.evidence_coverage:.1%}")
print()
print("Lightweight mode: ~65% catch rate, ~57% precision on HaluEval.")
print("Enable NLI (pip install director-ai[nli]) for 80%+ catch rate.")
print()
print("Competitive context (Feb 2026):")
cols = f"{'Tool':<25} {'Stream':>7} {'Custom KB':>10} {'Token-Level':>12}"
print(f"  {cols}")
print(f"  {'-' * 56}")
print(f"  {'Director-AI':<25} {'Yes':>7} {'Yes':>10} {'Yes':>12}")
print(f"  {'NeMo Guardrails':<25} {'No':>7} {'Partial':>10} {'No':>12}")
print(f"  {'Guardrails-AI':<25} {'No':>7} {'No':>10} {'No':>12}")
print(f"  {'LLM-Guard':<25} {'No':>7} {'No':>10} {'No':>12}")
print(f"  {'SelfCheckGPT':<25} {'No':>7} {'No':>10} {'No':>12}")
print()
print("Director-AI is the only guardrail with live mid-stream intervention")
print("and custom knowledge grounding.")

In [ ]:
import json

results_dir = project_root / "benchmarks" / "results"
results_dir.mkdir(parents=True, exist_ok=True)

output = {
    "benchmark": "E2E-Guardrail-Notebook",
    "director_ai_version": director_ai.__version__,
    "samples_per_task": SAMPLES_PER_TASK,
    "lightweight": metrics_light.to_dict(),
    "threshold_sweep": sweep_results,
    "optimal_threshold": best_thresh,
    "truthfulqa": {
        "total": tqa_result.total,
        "correct": tqa_result.correct,
        "accuracy": round(tqa_result.accuracy, 4),
    },
    "fallback_comparison": {
        name: m.to_dict() for name, m in mode_results.items()
    },
}

out_path = results_dir / "e2e_notebook_results.json"
out_path.write_text(json.dumps(output, indent=2), encoding="utf-8")
print(f"Full results saved to {out_path}")